# Limpieza y transformación de los datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('../df_cleaned.csv')

In [ ]:
df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
df.drop('index', axis=1, inplace=True)

In [ ]:
df.replace({'<NA>':np.nan}, inplace=True)

In [ ]:
# Definir columnas numéricas y categóricas
numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'label']

# Convertir '<NA>' a NaN
df.replace('<NA>', np.nan, inplace=True)

# Convertir 0.0 a NaN en la columna "chol"
df['chol'] = df['chol'].replace(0.0, np.nan)

# Reemplazar los valores 0.0 y los valores negativos por NaN en la columna 'oldpeak'
df['oldpeak'] = np.where((df['oldpeak'] <= 0), np.nan, df['oldpeak'])

# Convertir columnas numéricas a tipo float
df[numeric_cols] = df[numeric_cols].astype(float)

# Convertir columnas categóricas a tipo categórico
df[categorical_cols] = df[categorical_cols].astype('category')

# Imputación para variables numéricas (usando la media)
numeric_imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

# Imputación para variables categóricas (usando la moda)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Verificar que ya no hay valores faltantes
print(df.isnull().sum())

In [ ]:
# Variables tratadas con one-hot encoding
categorical_variables = ['cp', 'restecg', 'slope', 'thal', 'ca']

# Aplicar one-hot encoding a las variables categóricas
df = pd.get_dummies(df, columns=categorical_variables, drop_first=True)

# Mostrar el DataFrame con las variables codificadas
print(df.head())

In [ ]:
# Selecciona solo las características numéricas
numeric_data = df[['trestbps', 'chol', 'thalach', 'oldpeak', 'age']]

# Inicializa el escalador Min-Max
scaler = MinMaxScaler()

# Escala las características numéricas
scaled_numeric_data = scaler.fit_transform(numeric_data) 

# Crea un nuevo DataFrame con las características escaladas
scaled_data = pd.DataFrame(scaled_numeric_data, columns=['trestbps', 'chol', 'thalach', 'oldpeak', 'age'])

# Reemplaza las características originales con las características escaladas en tu DataFrame
df[['trestbps', 'chol', 'thalach', 'oldpeak', 'age']] = scaled_data

In [ ]:
df.head()

In [ ]:
# Separación de train y test
train_end = df[0:732]
test_end = df[(916-184):]

# PRUEBAS DE LOS MODELOS

## 4 TRY - Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Identificar la variable objetivo y las características para el entrenamiento
target_column = 'label'  # Nombre real de la variable objetivo
features_train = train_end.drop(columns=[target_column])
target_train = train_end[target_column]

# Identificar las características para la predicción en el conjunto de prueba
features_test = test_end.drop(columns=[target_column])

# Inicializar y entrenar el modelo de Random Forest para clasificación
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(features_train, target_train)

# Predecir las clases de las variables categóricas faltantes en el conjunto de prueba
predicted_classes = model.predict(features_test)

# Crear un nuevo DataFrame con las clases predichas
predicted_df = test_end.copy()
predicted_df[target_column] = predicted_classes

# Imprimir el DataFrame con las clases predichas
print(predicted_df)


In [ ]:
predicted_df['index'] = predicted_df.index

In [ ]:
predicted_df.reset_index(inplace=True)

In [ ]:
predicted_df.drop(columns=['index'], inplace=True)

# Renombrar la columna 'level_0' a 'index'
predicted_df.rename(columns={'level_0': 'index'}, inplace=True)

predicted_df['ID'] = range(len(predicted_df))
# Guardar las columnas 'index' y 'label' en un archivo CSV
predicted_df[['ID', 'label']].to_csv('try4_data.csv', index=False)

In [ ]:
predicted_df

In [ ]:
try4_df = pd.read_csv('try4_data.csv')

In [ ]:
try4_df

In [ ]:
try4_df['label'].value_counts()

## 5 TRY - SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Crear un modelo de clasificador SVM
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0, random_state=42))

# Entrenar el modelo SVM
svm_model.fit(features_train, target_train)

# Predecir las clases en el conjunto de prueba
predicted_classes_svm = svm_model.predict(features_test)

# Crear un nuevo DataFrame con las clases predichas por SVM
predicted_df_svm = test_end.copy()
predicted_df_svm[target_column] = predicted_classes_svm

# Imprimir el DataFrame con las clases predichas por SVM
print(predicted_df_svm)


In [ ]:
predicted_df_svm['index'] = predicted_df.index
predicted_df_svm.reset_index(inplace=True)

In [ ]:
predicted_df_svm.drop(columns=['index'], inplace=True)

# Renombrar la columna 'level_0' a 'index'
predicted_df_svm.rename(columns={'level_0': 'index'}, inplace=True)

predicted_df_svm['ID'] = range(len(predicted_df_svm))
# Guardar las columnas 'index' y 'label' en un archivo CSV
predicted_df_svm[['ID', 'label']].to_csv('try5_data.csv', index=False)

In [ ]:
try5_df = pd.read_csv('try5_data.csv')

In [ ]:
try5_df

In [ ]:
try5_df['label'].value_counts()

## 9 TRY - Random Forest

### Fue un error de try, se me olvidó escalar la edad.

## 10 TRY - Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Identificar la variable objetivo y las características para el entrenamiento
target_column = 'label'  # Nombre real de la variable objetivo
features_train = train_end.drop(columns=[target_column])
target_train = train_end[target_column]

# Identificar las características para la predicción en el conjunto de prueba
features_test = test_end.drop(columns=[target_column])

# Inicializar y entrenar el modelo de Random Forest para clasificación
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(features_train, target_train)

# Predecir las clases de las variables categóricas faltantes en el conjunto de prueba
predicted_classes = model.predict(features_test)

# Crear un nuevo DataFrame con las clases predichas
predicted_df = test_end.copy()
predicted_df[target_column] = predicted_classes

# Imprimir el DataFrame con las clases predichas
print(predicted_df)

In [ ]:
predicted_df['index'] = predicted_df.index

In [ ]:
predicted_df.reset_index(inplace=True)

In [ ]:
predicted_df.drop(columns=['index'], inplace=True)

# Renombrar la columna 'level_0' a 'index'
predicted_df.rename(columns={'level_0': 'index'}, inplace=True)

predicted_df['ID'] = range(len(predicted_df))
# Guardar las columnas 'index' y 'label' en un archivo CSV
predicted_df[['ID', 'label']].to_csv('try44_data.csv', index=False)

In [ ]:
try44_df = pd.read_csv('try44_data.csv')

In [ ]:
try44_df['label'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

# Variable chol
chol_values = df['chol']

# Crear el histograma
plt.figure(figsize=(8, 6))
plt.hist(chol_values, bins=30, color='skyblue', edgecolor='black')
plt.title('Distribución de la variable "chol"')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()


## 16 TRY - Random Forest

In [ ]:
# Convertir '<NA>' a NaN
df.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols_mean = ['trestbps', 'thalach', 'chol']
numeric_cols_median = ['oldpeak', 'age']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal', 'label', 'ca']

# Imputación para variables numéricas con la media
numeric_imputer_mean = SimpleImputer(strategy='mean')
df[numeric_cols_mean] = numeric_imputer_mean.fit_transform(df[numeric_cols_mean])

# Imputación para variables numéricas con la mediana
numeric_imputer_median = SimpleImputer(strategy='median')
df[numeric_cols_median] = numeric_imputer_median.fit_transform(df[numeric_cols_median])

# Imputación para variables categóricas (usando la moda)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Verificar que ya no hay valores faltantes
print(df.isnull().sum())

## 17 TRY - Random Forest

In [ ]:
# Convertir '<NA>' a NaN
df.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols_mean = ['trestbps', 'thalach', 'age']
numeric_cols_median = ['oldpeak', 'chol']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal', 'label', 'ca']

# Imputación para variables numéricas con la media
numeric_imputer_mean = SimpleImputer(strategy='mean')
df[numeric_cols_mean] = numeric_imputer_mean.fit_transform(df[numeric_cols_mean])

# Imputación para variables numéricas con la mediana
numeric_imputer_median = SimpleImputer(strategy='median')
df[numeric_cols_median] = numeric_imputer_median.fit_transform(df[numeric_cols_median])

# Imputación para variables categóricas (usando la moda)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Verificar que ya no hay valores faltantes
print(df.isnull().sum())

## 18 TRY - Random Forest

In [ ]:
# Convertir '<NA>' a NaN
df.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols_mean = ['trestbps', 'thalach', 'age', 'chol']
numeric_cols_median = ['oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal', 'label', 'ca']

# Imputación para variables numéricas con la media
numeric_imputer_mean = SimpleImputer(strategy='mean')
df[numeric_cols_mean] = numeric_imputer_mean.fit_transform(df[numeric_cols_mean])

# Imputación para variables numéricas con la mediana
numeric_imputer_median = SimpleImputer(strategy='median')
df[numeric_cols_median] = numeric_imputer_median.fit_transform(df[numeric_cols_median])

# Imputación para variables categóricas (usando la moda)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Verificar que ya no hay valores faltantes
print(df.isnull().sum())

## 21 TRY - XG BOOST

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Dividir los datos en características (X) y etiquetas (y)
X_train = train_end.drop(columns=['label'])
y_train = train_end['label']
X_test = test_end.drop(columns=['label'])
y_test = test_end['label']

# Convertir los datos al formato DMatrix requerido por XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definir los parámetros del modelo XGBoost
params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': len(df['label'].unique()),  # Número de clases
    'max_depth': 6,  # Profundidad máxima del árbol
    'eta': 0.3,  # Tasa de aprendizaje
    'subsample': 0.8,  # Submuestreo de filas
    'colsample_bytree': 0.8,  # Submuestreo de columnas
    'eval_metric': 'merror'  # Métrica de evaluación
}

# Entrenar el modelo XGBoost
num_round = 100  # Número de rondas de entrenamiento
model = xgb.train(params, dtrain, num_round)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(dtest)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo XGBoost:", accuracy)